In [1]:
!pip install crfm-helm

  Obtaining dependency information for crfm-helm from https://files.pythonhosted.org/packages/cb/ac/a22f4d26b7784bf896a1e9d1b00b152fa0102e3646c8169336a5ac8a6b63/crfm_helm-0.5.5-py3-none-any.whl.metadata
  Obtaining dependency information for cattrs~=22.2 from https://files.pythonhosted.org/packages/43/3b/1d34fc4449174dfd2bc5ad7047a23edb6558b2e4b5a41b25a8ad6655c6c7/cattrs-22.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for dacite~=1.6 from https://files.pythonhosted.org/packages/94/35/386550fd60316d1e37eccdda609b074113298f23cef5bddb2049823fe666/dacite-1.9.2-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-resources~=5.10 from https://files.pythonhosted.org/packages/7a/68/bd9dd6bbf06772c7accce77d0354d783333fbe712a60b08fc13540c05422/importlib_resources-5.13.0-py3-none-any.whl.metadata
  Obtaining dependency information for Mako~=1.2 from https://files.pythonhosted.org/packages/cd/83/de0a49e7de540513f53ab5d2e105321dedeb08a8f5850f0208decf4390ec/

In [10]:
!pip install "crfm-helm[all]"

  Obtaining dependency information for pydub~=0.25.1 from https://files.pythonhosted.org/packages/a6/53/d78dc063216e62fc55f6b2eebb447f6a4b0a59f55c8406376f76bf959b08/pydub-0.25.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for ffmpeg-python~=0.2.0 from https://files.pythonhosted.org/packages/d7/0c/56be52741f75bad4dc6555991fabd2e07b432d333da82c11ad701123888a/ffmpeg_python-0.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for soundfile~=0.12 from https://files.pythonhosted.org/packages/a0/e5/58fd1a8d7b26fc113af244f966ee3aecf03cb9293cb935daaddc1e455e18/soundfile-0.13.1-py2.py3-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for librosa~=0.10 from https://files.pythonhosted.org/packages/b5/ba/c63c5786dfee4c3417094c4b00966e61e4a63efecee22cb7b4c0387dda83/librosa-0.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for einops~=0.7.0 from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212

In [1]:
from helm.benchmark.metrics.vision_language.image_metrics import AnnotatedImageMetrics
from typing import Dict, Callable, Generator, Mapping, Tuple

class Cache(object):
    def get(self, request: Mapping, compute: Callable[[], Dict]) -> Tuple[Dict, bool]:
        response = compute()
        return response, False

metrics = AnnotatedImageMetrics(generation_type="webpage", metric_names=['ems'])
metrics._cache = Cache()

/Users/tcz/Dropbox/Reverse Browser/HelmTest/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from PIL import Image
from tqdm import tqdm

image1 = Image.open('../Figures/checkerboard1.png')
image2 = Image.open('../Figures/checkerboard2.png')

image3 = Image.open('../Figures/lipsum1.png')
image4 = Image.open('../Figures/lipsum2.png')

patch_sizes = [
    (5, 5),
    (6, 6),
    (7, 7),
    (8, 8),
    (9, 9),
    (10, 10),
]

for patch_size in patch_sizes:
    ems1 = metrics.ems(image1, image2, patch_size=patch_size)
    ems2 = metrics.ems(image3, image4, patch_size=patch_size)

    ems1 = round(ems1, 6)
    ems2 = round(ems2, 6)

    print(f"{patch_size} & {ems1} & {ems2} \\\\")

(5, 5) & 0.624331 & 0.912573 \\
(6, 6) & 0.713801 & 0.942947 \\
(7, 7) & 0.477627 & 0.913324 \\
(8, 8) & 0.540136 & 0.888891 \\
(9, 9) & 0.780074 & 0.888672 \\
(10, 10) & 0.830694 & 0.882436 \\


In [9]:


for patch_size in tqdm(patch_sizes):
    ems = metrics.ems(image1, image2, patch_size=patch_size)
    print(f"Patch size {patch_size}, {ems}")

 17%|█▋        | 1/6 [00:00<00:03,  1.35it/s]

Patch size (5, 5), 0.9125733811113541


 33%|███▎      | 2/6 [00:02<00:05,  1.40s/it]

Patch size (6, 6), 0.9429469974308775


 50%|█████     | 3/6 [00:07<00:09,  3.04s/it]

...

Patch size (8, 8), 0.8888912995890974


 83%|████████▎ | 5/6 [00:46<00:13, 13.95s/it]

Patch size (9, 9), 0.888672081466845


100%|██████████| 6/6 [01:40<00:00, 16.79s/it]

Patch size (10, 10), 0.8824359543061934


In [37]:
import os
import re
from PIL import Image
import numpy as np
from tqdm import tqdm

# Path to the directory containing the images (adjust if needed)
directory = '../Models/helm-generations/screenshots'

# Regex to extract identifier and type from file names
file_pattern = re.compile(r'^([0-9a-f]{32})_(expected|predicted)\.png$')

# Group files by their identifier
pairs = {}
for filename in os.listdir(directory):
    match = file_pattern.match(filename)
    if match:
        ident, img_type = match.groups()
        if ident not in pairs:
            pairs[ident] = {}
        pairs[ident][img_type] = os.path.join(directory, filename)

print(f"Found {len(pairs)} pairs of images.")

# List to store EMS scores
ems_scores = []

# Process each pair that has both images
for ident, files in tqdm(pairs.items()):
    if 'expected' in files and 'predicted' in files:
        # Open images using PIL.Image.open
        expected_image = Image.open(files['expected'])
        predicted_image = Image.open(files['predicted'])
        # Calculate EMS score with the predicted image as first argument
        try:
            score = metrics.ems(predicted_image, expected_image)
        except ZeroDivisionError as e:
            score = 1.0
        except Exception as e:
            print(f"Error processing {ident}: {e}")
            continue
        ems_scores.append(score)
    else:
        print(f"Missing images for identifier: {ident}")

# Calculate and print the average EMS score
if ems_scores:
    average_score = np.mean(ems_scores)
    print("Valid pairs:", len(ems_scores))
    print("Average EMS:", average_score)
else:
    print("No valid image pairs found.")

Found 883 pairs of images.


 70%|███████   | 621/883 [1:36:55<34:35,  7.92s/it]  

Missing images for identifier: d9c042cbaf73420ab27b053d48f6a181


100%|██████████| 883/883 [2:14:24<00:00,  9.13s/it]

Valid pairs: 882
Average EMS: 0.6936815129714871


In [17]:
%%time
from tqdm import tqdm

for ident, files in tqdm(list(pairs.items())[:10]):
    expected_image = Image.open(files['expected'])
    predicted_image = Image.open(files['predicted'])
    metrics.ems(expected_image, predicted_image)

100%|██████████| 10/10 [00:53<00:00,  5.31s/it]

CPU times: user 52.6 s, sys: 292 ms, total: 52.9 s
Wall time: 53.1 s


In [2]:
import torch
import PIL
import numpy as np
import lpips
import torchvision.transforms as transforms

PIL.Image.MAX_IMAGE_PIXELS = 1000000000

server_thread = None

def remove_alpha(image):
    if image.shape[0] == 4:
        return image[:3, :, :]
    return image

def calculate_mse(image1, image2):
    return ((image1 - image2) ** 2).mean().item()

def resize_image(image):
    return torch.nn.functional.avg_pool2d(image, 2)

def preprocess(image1, image2):
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    transform = transforms.ToTensor()

    if image1.size == image2.size:
        new_image2 = Image.new("RGB", image1.size, (255, 255, 255))
        new_image2.paste(image2, (0, 0))
        image2 = new_image2.crop((0, 0, image1.size[0], image1.size[1]))

    image1 = remove_alpha(transform(image1)).to(device)
    image2 = remove_alpha(transform(image2)).to(device)

    return image1, image2

def calculate_similarity(image1, image2):
    image1, image2 = preprocess(image1, image2)
    errors = []
    while True:
        mse = calculate_mse(image1, image2)

        errors.append(mse)

        _, h, w = image1.size()
        if h == 1 or w == 1:
            break

        image1 = resize_image(image1)
        image2 = resize_image(image2)

    average_mse = np.mean(errors)
    sim = 1 - average_mse

    return float(sim)

def calculate_perceptual_loss(image1, image2):
    image1, image2 = preprocess(image1, image2)
    loss_fn_alex = lpips.LPIPS(net='alex', verbose=False)
    loss_fn_alex.to(image1.device)

    # Normalize to [-1, 1]
    image1 = (image1 - 0.5) * 2
    image2 = (image2 - 0.5) * 2

    loss = loss_fn_alex(image1, image2)

    return loss.squeeze().item()

def calculate_perceptual_loss_vgg(image1, image2):
    image1, image2 = preprocess(image1, image2)
    loss_fn_vgg = lpips.LPIPS(net='vgg', verbose=False)
    loss_fn_vgg.to(image1.device)

    # Normalize to [-1, 1]
    image1 = (image1 - 0.5) * 2
    image2 = (image2 - 0.5) * 2

    loss = loss_fn_vgg(image1, image2)

    return loss.squeeze().item()


In [24]:
%%time
from tqdm import tqdm

for ident, files in tqdm(list(pairs.items())[:10]):
    expected_image = Image.open(files['expected'])
    predicted_image = Image.open(files['predicted'])
    calculate_similarity(expected_image, predicted_image)

100%|██████████| 10/10 [00:01<00:00,  7.56it/s]

CPU times: user 2.29 s, sys: 883 ms, total: 3.17 s
Wall time: 1.33 s


In [25]:
%%time
from tqdm import tqdm

for ident, files in tqdm(list(pairs.items())[:10]):
    expected_image = Image.open(files['expected'])
    predicted_image = Image.open(files['predicted'])
    calculate_perceptual_loss(expected_image, predicted_image)

  0%|          | 0/10 [00:00<?, ?it/s]/Users/tcz/Dropbox/Reverse Browser/HelmTest/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tcz/Dropbox/Reverse Browser/HelmTest/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/tcz/Dropbox/Reverse Browser/HelmTest/venv/lib/python3.9/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to constr

CPU times: user 7.59 s, sys: 2.77 s, total: 10.4 s
Wall time: 4.49 s


In [31]:
%%time
from tqdm import tqdm


blank_image = Image.open('blank.png')
blank_image2 = Image.open('blank.png')
try:
    metrics.ems(blank_image, blank_image2)
except ZeroDivisionError as e:
    print("ZeroDivisionError:", e)

ZeroDivisionError: division by zero
CPU times: user 7.71 ms, sys: 8.58 ms, total: 16.3 ms
Wall time: 15.9 ms


In [3]:
import os
import re
from PIL import Image
import numpy as np
from tqdm import tqdm

# Path to the directory containing the images (adjust if needed)
directory = '../Models/helm-generations/screenshots'

# Regex to extract identifier and type from file names
file_pattern = re.compile(r'^([0-9a-f]{32})_(expected|predicted)\.png$')

# Group files by their identifier
pairs = {}
for filename in os.listdir(directory):
    match = file_pattern.match(filename)
    if match:
        ident, img_type = match.groups()
        if ident not in pairs:
            pairs[ident] = {}
        pairs[ident][img_type] = os.path.join(directory, filename)

print(f"Found {len(pairs)} pairs of images.")

# List to store EMS scores
vgg_scores = []

# Process each pair that has both images
for ident, files in tqdm(pairs.items()):
    if 'expected' in files and 'predicted' in files:
        # Open images using PIL.Image.open
        expected_image = Image.open(files['expected'])
        predicted_image = Image.open(files['predicted'])
        # Calculate EMS score with the predicted image as first argument
        try:
            score = calculate_perceptual_loss_vgg(predicted_image, expected_image)
        except ZeroDivisionError as e:
            score = 1.0
        except Exception as e:
            print(f"Error processing {ident}: {e}")
            continue
        vgg_scores.append(score)
    else:
        print(f"Missing images for identifier: {ident}")

# Calculate and print the average EMS score
if vgg_scores:
    average_score = np.mean(vgg_scores)
    print("Valid pairs:", len(vgg_scores))
    print("Average LPIPS (VGG):", average_score)
    print("Average LPIPS (VGG, normalized):", 1 - average_score)
else:
    print("No valid image pairs found.")

Found 883 pairs of images.


  0%|          | 0/883 [00:00<?, ?it/s]/Users/tcz/Dropbox/Reverse Browser/HelmTest/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tcz/Dropbox/Reverse Browser/HelmTest/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/tcz/Dropbox/Reverse Browser/HelmTest/venv/lib/python3.9/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct

Missing images for identifier: d9c042cbaf73420ab27b053d48f6a181


100%|██████████| 883/883 [32:30<00:00,  2.21s/it]


NameError: name 'ems_scores' is not defined

In [4]:
average_score = np.mean(vgg_scores)
print("Valid pairs:", len(vgg_scores))
print("Average LPIPS (VGG):", average_score)
print("Average LPIPS (VGG, normalized):", 1 - average_score)


Valid pairs: 882
Average LPIPS (VGG): 0.24492059239961328
Average LPIPS (VGG, normalized): 0.7550794076003867
